# LastFM Recommender
The aim of this exercies is to build an ensemble recommender for music artists, using data made available [here](http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz).

I'll follow the methods described in [Jeremy Howards video](https://www.youtube.com/watch?v=V2h3IOBDvrA&t=5761s).

This data set is a single table with 350k users organised into the following rows:
- UserID 
- ArtistID 
- ArtistName 
- PlayCount

Not all of the artists have a valid ArtistID, so we will use the artist name as a unique id.

In [1]:
from theano.sandbox import cuda
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN not available)
Using Theano backend.


In [2]:
path = 'data/lastfm-dataset-360K/'
fulldata_file = 'usersha1-artmbid-artname-plays.tsv'
data_file = 'fulldata.tsv'
sample_file = 'sampledata.tsv'

def read_fulldata_file():
    return pd.read_csv(path + fulldata_file, 
                       sep='\t',
                       usecols=[0,2,3],
                       names=['user', 'artist','plays'])

def read_sample_file():
    return pd.read_csv(path + sample_file,
                       sep='\t')

# Create the sample dataset of 1000 users
if not os.path.isfile(path + sample_file):
    df = read_fulldata_file()
    users_to_sample = df.user.sample(n=5000)
    rows_to_sample = df[df.user.isin(users_to_sample)]
    rows_to_sample.to_csv(path + sample_file,
                          index=False,
                          sep='\t')

In [3]:
df = read_sample_file()
# df = read_fulldata_file()

First, we need to transform the data somewhat:
- UserID and Artist name to continguous integers
- Playcount value for each (user, artist) tuple into a normalized value representing how much the user likes that artist compared to other artists that they have listened to.

We shall assign each (user, artist) tuple a value representing the fraction of all of that users plays that the artist represents. This should then leave the value normalized between 0 and 1.

In [4]:
userid2ids = {o:i for i,o in enumerate(df.user.unique())}
artistid2ids = {o:i for i,o in enumerate(df.artist.unique())}
total_plays_per_user = df.groupby(['user'])['plays'].sum().to_dict()

def normalize(row):
    row['plays'] = row['plays'] / total_plays_per_user[row['user']]
    row['user'] = userid2ids[row['user']]
    row['artist'] = artistid2ids[row['artist']]
    return row

norm_df = df.apply(normalize, axis=1)

Now we can decide on a number of latent factors and split it out into training and validation sets. We also create a few variables that we will need later.

In [5]:
n_factors = 40
np.random.seed = 42
msk = np.random.rand(len(norm_df)) < 0.8
trn = norm_df[msk]
val = norm_df[~msk]

n_users = norm_df.user.nunique()
n_artists = norm_df.artist.nunique()
n_users, n_artists

(4969, 39217)

As per the original example, we'll do a quick cross tab table of the top artists and most prolific users to sanity check how we are doing so far.

In [6]:
g_artists = norm_df.groupby('artist')['plays'].count()
top_artists = g_artists.sort_values(ascending=False)[:15]
g_users = norm_df.groupby('user')['artist'].count()
top_users = g_users.sort_values(ascending=False)[:15]

top = norm_df.join(top_users, rsuffix='_r', how='inner', on='user')
top = top.join(top_artists, rsuffix='_r', how='inner', on='artist')
pd.crosstab(top.user, top.artist, top.plays, aggfunc = np.sum)

artist,110,125,142,152,200,210,225,230,233,242,597,1029,1863
user,,,,,,,,,,,,,
588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003513,0.000567,0.009803,0.012070
1295,NaN,NaN,NaN,NaN,0.005072,NaN,NaN,NaN,NaN,NaN,0.042796,NaN,NaN
2186,NaN,NaN,NaN,0.009560,NaN,NaN,NaN,0.047270,NaN,NaN,NaN,0.029743,NaN
2350,0.001881,NaN,NaN,0.002796,0.012480,NaN,0.026332,0.049207,NaN,NaN,NaN,0.065194,NaN
2811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087297,NaN,NaN,NaN,NaN,NaN
3499,NaN,NaN,0.012577,NaN,NaN,NaN,NaN,NaN,0.008724,0.090731,NaN,NaN,NaN
3689,0.026765,0.013753,NaN,NaN,0.005364,NaN,NaN,0.005878,NaN,0.019289,0.069680,NaN,0.010615
4002,NaN,NaN,NaN,NaN,NaN,0.000672,NaN,0.000532,NaN,NaN,NaN,NaN,NaN
4247,NaN,0.016451,NaN,NaN,NaN,NaN,NaN,0.012010,NaN,NaN,NaN,NaN,NaN


The resulting table is a lot more sparse than the equivilent table for movie titles. I'm guessing this is because there is a much larger number of distinct artists than there are movies compared to the overall size of the dataset.

# Dot product
The most basic model as per the original example.

In [7]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
artist_in = Input(shape=(1,), dtype='int64', name='artist_in')
a = Embedding(n_artists, n_factors, input_length=1, W_regularizer=l2(1e-4))(artist_in)

In [8]:
x = merge([u, a], mode='dot')
x = Flatten()(x)
model = Model([user_in, artist_in], x)
model.compile(Adam(0.001), loss='mse')

In [9]:
model.fit([trn.user, trn.artist], 
          trn.plays, 
          batch_size=64, 
          nb_epoch=2, 
          validation_data=([val.user, val.artist], val.plays))

Train on 200214 samples, validate on 49997 samples
Epoch 1/2
200214/200214 [==============================] - 24s - loss: 0.0015 - val_loss: 9.0932e-04
Epoch 2/2
200214/200214 [==============================] - 24s - loss: 8.5566e-04 - val_loss: 9.0932e-04


In [10]:
model.optimizer.lr = 0.01
model.fit([trn.user, trn.artist], 
          trn.plays, 
          batch_size=64, 
          nb_epoch=3,
          validation_data=([val.user, val.artist], val.plays))

Train on 200214 samples, validate on 49997 samples
Epoch 1/3
200214/200214 [==============================] - 24s - loss: 8.5566e-04 - val_loss: 9.0932e-04
Epoch 2/3
200214/200214 [==============================] - 24s - loss: 8.5566e-04 - val_loss: 9.0932e-04
Epoch 3/3
200214/200214 [==============================] - 24s - loss: 8.5566e-04 - val_loss: 9.0932e-04


In [11]:
model.optimizer.lr = 0.001
model.fit([trn.user, trn.artist], 
          trn.plays, 
          batch_size=64, 
          nb_epoch=3,
          validation_data=([val.user, val.artist], val.plays))

Train on 200214 samples, validate on 49997 samples
Epoch 1/3
200214/200214 [==============================] - 24s - loss: 8.5567e-04 - val_loss: 9.0934e-04
Epoch 2/3
200214/200214 [==============================] - 24s - loss: 8.5568e-04 - val_loss: 9.0933e-04
Epoch 3/3
200214/200214 [==============================] - 24s - loss: 8.5568e-04 - val_loss: 9.0933e-04


In [12]:
model.predict([np.array([588]), np.array([242])])
# Should be roughly 0.003513

array([[ -1.5190e-12]], dtype=float32)

# Bias
Something that we don't account for are users that listen to an extremely broad range of music. This would appear to give all of their artists lower ratings than other users. A user bias value might help this.
I'm going to follow the course example and also assign a bias value to the artists as well.

This is represented as a single value for each user and a single value for each artist.

In [13]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [14]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
artist_in, a = embedding_input('artist_in', n_artists, n_factors, 1e-4)

In [15]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [16]:
ub = create_bias(user_in, n_users)
ab = create_bias(artist_in, n_artists)

In [17]:
x = merge([u, a], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, ab], mode='sum')
model = Model([user_in, artist_in], x)
model.compile(Adam(0.001), loss='mse')

In [18]:
model.fit([trn.user, trn.artist], 
          trn.plays, 
          batch_size=64, 
          nb_epoch=1, 
          validation_data=([val.user, val.artist], val.plays))

Train on 200214 samples, validate on 49997 samples
Epoch 1/1
200214/200214 [==============================] - 26s - loss: 0.0016 - val_loss: 7.6381e-04


In [ ]:
model.optimizer.lr = 0.01
model.fit([trn.user, trn.artist], 
          trn.plays, 
          batch_size=64, 
          nb_epoch=3,
          validation_data=([val.user, val.artist], val.plays))

Train on 200214 samples, validate on 49997 samples
Epoch 1/3
200214/200214 [==============================] - 26s - loss: 6.4063e-04 - val_loss: 7.7936e-04
Epoch 2/3
200214/200214 [==============================] - 25s - loss: 6.2755e-04 - val_loss: 7.9021e-04
Epoch 3/3
200214/200214 [==============================] - 26s - loss: 6.3193e-04 - val_loss: 7.9007e-04


In [ ]:
model.optimizer.lr = 0.001
model.fit([trn.user, trn.artist], 
          trn.plays, 
          batch_size=64, 
          nb_epoch=3,
          validation_data=([val.user, val.artist], val.plays))

Train on 200214 samples, validate on 49997 samples
Epoch 1/3
200214/200214 [==============================] - 26s - loss: 6.2393e-04 - val_loss: 7.9219e-04
Epoch 2/3
185088/200214 [==========================>...] - ETA: 1s - loss: 6.2200e-04

Either these results are really good or something has gone wrong. Either way, we are overfitting. We shall save the weights and try a CNN.

In [ ]:
model.save_weights(path + 'bias.h5')
model.load_weights(path + 'bias.h5')

In [ ]:
model.predict([np.array([588]), np.array([242])])
# Should be roughly 0.003513